In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
-- BLOCK đích danh STORE, SKU
SELECT * EXCEPT (proc_date, upload_time ,upload_user)
FROM {catalog_name}.udp_wcm_silver_dataportal.d_cfg_winare_block_by_store_sku
QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id, product_id, reason ORDER BY proc_date DESC, upload_time DESC) = 1
UNION ALL
-- BLOCK theo giỏ hàng
SELECT DISTINCT
	a.store_id,
	a.product_id,
	"Block giỏ hàng" reason,
	DATE("2000-01-01") as valid_from,
	DATE("2030-12-31") as valid_to
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a
WHERE
	a.assortment IN (SELECT assortment_basket FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_block_by_assortment_basket)
  	AND a.status = 'ACTIVE'
  	AND (DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) >= a.start_date AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.end_date);
""")

In [0]:
spark.sql("""
DELETE FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku_current_his
WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1;
""")

In [0]:
spark.sql("""
INSERT INTO {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku_current_his
SELECT
	DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 calday,
	store_id,
	product_id, 
	reason,
	`from`,
	`to`
FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku_current_his
WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN `from` AND `to`;
""")